## Text Translation and Sentiment Analysis using Transformers

### Project Overview:
The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named movie_reviews_eng.csv, movie_reviews_fr.csv, and movie_reviews_sp.csv.

  -  The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.
  
  - Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.
  
  - Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows

The output of the project will be a CSV file with a header row that includes column names such as Title, Year, Synopsis, Review, Review Sentiment, and Original Language. The Original Language column will indicate the language of the review and synopsis (en/fr/sp) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/Users/chibuzor/Downloads/Python Repo/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data():
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("/Users/chibuzor/Downloads/Python Repo/Sentiment-Analysis_for_Movies_with_Transformers/Data/movie_reviews_eng.csv")
    df_esp = pd.read_csv("/Users/chibuzor/Downloads/Python Repo/Sentiment-Analysis_for_Movies_with_Transformers/Data/movie_reviews_sp.csv")
    df_fr = pd.read_csv("/Users/chibuzor/Downloads/Python Repo/Sentiment-Analysis_for_Movies_with_Transformers/Data/movie_reviews_fr.csv")
    
    df_eng["Original Language"] = "English"
    df_esp["Original Language"] = "Spanish"
    df_fr["Original Language"] = "French"
    
    df_eng.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    df_esp.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    
    # Concatenating the dataframes
    df = pd.concat([df_eng, df_esp, df_fr], ignore_index=True)
    df = df.rename(columns={'Movie / TV Series': 'Title'})
    return df

df = preprocess_data()

In [3]:
df.head()

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English


In [4]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
24,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
10,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Spanish
14,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
19,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",Spanish
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
16,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
27,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French


#### Text translation
##### Translate the <b>Review</b> and <b>Synopsis</b> column values to English.

In [ ]:
# load translation models and tokenizers
import warnings
warnings.filterwarnings('ignore')

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_model = MarianTokenizer.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianMTModel.from_pretrained(es_en_model_name)


def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    #inputs = tokenizer(text, return_tensors="pt")
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [6]:
# Filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == 'fr']['Review'] #"fr" represents the French language
fr_reviews_en = fr_reviews.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == 'fr']['Synopsis'] 
fr_synopsis_en = fr_synopsis.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'es']['Review'] 
es_reviews_en = es_reviews.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == 'es']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))

# Update the DataFrame with translated French reviews and synopses
df.loc[df['Original Language'] == 'fr', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'fr', 'Synopsis'] = fr_synopsis_en

df.loc[df['Original Language'] == 'es', 'Review'] = es_reviews_en
df.loc[df['Original Language'] == 'es', 'Synopsis'] = es_synopsis_en

#### Sentiment Analysis
##### Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result Positive or Negative in a new column titled Sentiment in the dataframe.

In [7]:
# Specify the model name
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load the sentiment analysis pipeline with the specified model
sentiment_classifier = pipeline('sentiment-analysis', model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    classified_text = classifier(text)
    classified_text = classified_text[0]['label'] #[1] gives the scores
    return classified_text

Device set to use mps:0


In [8]:
# Create a new column 'Sentiment' in the DataFrame
df['Sentiment'] = df['Review'].apply(lambda review: analyze_sentiment(review, sentiment_classifier))

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
23,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French,POSITIVE
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,POSITIVE
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,NEGATIVE
12,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Spanish,NEGATIVE
26,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",French,NEGATIVE
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English,POSITIVE
20,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",French,POSITIVE
13,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish,NEGATIVE
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English,POSITIVE
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,POSITIVE


In [10]:
# export the results to a .csv file
import os

# Check if the directory exists, and if not, create it
directory = 'result'
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to a CSV file in the "result" directory
df.to_csv(os.path.join(directory, "reviews_with_sentiment.csv"), index=False)